In [1]:
import os
from subprocess import run
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# viz libraries
import vedo as vd
import pyvista as pv
from scipy.ndimage import distance_transform_edt 

#from surfrac import SurFrac
from pysimfrac import *
from MP_LBM_utils import write_MPLBM, check_lbm_install, read_permeability

# 1. Create a fracture surface

In [2]:
#myfrac = SimFrac( h = 1.0, lx = 512, ly = 128, method = "spectral")
myfrac = SimFrac( h = 1.0, lx = 128, ly = 64, method = "spectral")

myfrac.params['aniso']['value'] = 0.0
myfrac.params['H']['value'] = 0.7
myfrac.params['roughness']['value'] = 4
myfrac.params['mismatch']['value'] = 0.25
myfrac.params['lambda_0']['value'] = 0.6
myfrac.params['model']['value'] = 'smooth'
myfrac.params['seed']['value'] = 1
myfrac.create_fracture()
myfrac.set_mean_aperture(15)    

--> Checking spectral Method Parameters: Starting
--> Checking spectral Method Parameters: Complete
--> Running Fracture surface method 'spectral': Starting
--> Checking spectral Method Parameters: Starting
--> Checking spectral Method Parameters: Complete


--> Generation Method: spectral
 * Parameter Name: H
 * Value: 0.7
 * Description: Hurst exponent. Determines fractal dimension. Range is (0, 1)


 * Parameter Name: roughness
 * Value: 4
 * Description: Root-mean-squared value of heights [m], Range is (0, infty)


 * Parameter Name: mean-aperture
 * Value: 1
 * Description: Mean Fracture Aperture


 * Parameter Name: mismatch
 * Value: 0.25
 * Description: Mismatch length scale (wavelength) as a fraction of fracture size [0 < Mismatch < 1]


 * Parameter Name: N
 * Value: 128
 * Description: Discretization of fracture self.lx/self.h


 * Parameter Name: aniso
 * Value: 0.0
 * Description: Anisotropy Ratio [0 < Aniso < 1]. Setting to 0 is isotropic.


 * Parameter Name: seed
 * Val

# 2. Voxelize

In [3]:
myfrac.voxelize(solid_voxels=5) 
frac_3D = myfrac.frac_3D.transpose(2,0,1)

--> Checking aperture values
--> Complete
--> Setting minimum surface value of bottom to 0
--> Complete


In [18]:
frac_3D.shape

(49, 64, 128)

# 3. Visualize the 3D fracture

In [4]:
vd.settings.default_backend= 'vtk'
vdp = vd.Plotter(axes=9, bg='w', bg2='lightblue', size=(1200,900), offscreen=False)

frac_3D = myfrac.frac_3D.transpose(2,0,1)
edist = distance_transform_edt(frac_3D==0)
lego = vd.Volume(edist).legosurface(vmin=1, vmax=5).cmap('turbo',vmin=-1, vmax=5).lighting(ambient=2)
lego += vd.Volume((frac_3D==0)*1.0).legosurface(vmin=1, vmax=2).c('lightgray').opacity(0.05).lighting('shiny')

cam = dict(
            position=(452.134, 486.466, -150.875),
            focal_point=(-2.81867, 55.7670, 224.764),
            viewup=(0.782363, -0.468078, 0.410867),
            distance=730.471,
            clipping_range=(343.033, 1235.15),
            )

vdp.show( lego, camera=cam, axes=1).close()

# 4. Install LBM library

In [20]:
# !git clone git@github.com:je-santos/MPLBM-UT.git
# os.chdir('MPLBM-UT')
# os.system('bash Install.sh')
# os.chdir('..')

# 5. Check the LBM installation

In [19]:
check_lbm_install(sim_mode='two-phase')


LBM for two-phase mode installation was found



# 6. Create the single-phase LBM input deck

In [7]:
sim_max_iter=100000
lbm = write_MPLBM(
                  frac_obj = myfrac, 
                  buffer_layers = 2,
                  cpus = 4,
                  num_hrs = 4,
                  allocation = None,
                  sim_mode='two-phase'
                  )

write LBM end


# 7. Run the simulation

In [9]:
os.chdir(f'{lbm.folder_path}')
os.system('bash run_frac.sh')

0

# 8. Visualize the f1 density

In [17]:

# Load data from .dat file
data = np.loadtxt('output/rho_f1_000.dat')
data_reshaped = data.reshape(frac_3D.shape[0],frac_3D.shape[1],frac_3D.shape[2])

# Visualize the data
vel_thresholds = np.linspace(np.min(data_reshaped), np.max(data_reshaped), 40)
vel = vd.Volume(data_reshaped).isosurface(value=vel_thresholds)

vp = vd.Plotter(axes=9, bg='w', bg2='lightblue', size=(1200,900), offscreen=False)
vp += vel.cmap('turbo').lighting('glossy').opacity(0.6).add_scalarbar('f1 density', font_size=16).lighting(ambient=.25)


# Camera settings (adjust as needed)
cam = dict(
    position=(452.134, 486.466, -150.875),
    focal_point=(-2.81867, 55.7670, 224.764),
    viewup=(0.782363, -0.468078, 0.410867),
    distance=730.471,
    clipping_range=(343.033, 1235.15),
)

vd.show(camera=cam, axes=1).close()